In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, sum
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [2]:
spark = SparkSession.builder \
    .appName("feature-engineering") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

In [3]:
sales = spark.read.parquet("../dataset/cooked/preprocessed_sales")

In [4]:
sales.printSchema()
sales.show(5)

root
 |-- id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- cat_id: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- day: string (nullable = true)
 |-- sales: integer (nullable = true)

+--------------------+-----------+-------+------+--------+--------+---+-----+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|day|sales|
+--------------------+-----------+-------+------+--------+--------+---+-----+
|FOODS_1_177_CA_3_...|FOODS_1_177|FOODS_1| FOODS|    CA_3|      CA|d_1|    0|
|FOODS_1_177_CA_3_...|FOODS_1_177|FOODS_1| FOODS|    CA_3|      CA|d_2|    1|
|FOODS_1_177_CA_3_...|FOODS_1_177|FOODS_1| FOODS|    CA_3|      CA|d_3|    0|
|FOODS_1_177_CA_3_...|FOODS_1_177|FOODS_1| FOODS|    CA_3|      CA|d_4|    0|
|FOODS_1_177_CA_3_...|FOODS_1_177|FOODS_1| FOODS|    CA_3|      CA|d_5|    0|
+--------------------+-----------+-------+------+--------+--------+

# Improt CSV

In [5]:
calendar = spark.read.csv(
    "../dataset/raw/calendar.csv",
    header=True,
    inferSchema=True
)

sell_prices = spark.read.csv(
    "../dataset/raw/sell_prices.csv",
    header=True,
    inferSchema=True
)

In [13]:
calendar.show(), sell_prices.show()

+----------+--------+---------+----+-----+----+----+-------------+------------+------------+------------+-------+-------+-------+
|      date|wm_yr_wk|  weekday|wday|month|year|   d| event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|
+----------+--------+---------+----+-----+----+----+-------------+------------+------------+------------+-------+-------+-------+
|2011-01-29|   11101| Saturday|   1|    1|2011| d_1|         NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-01-30|   11101|   Sunday|   2|    1|2011| d_2|         NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-01-31|   11101|   Monday|   3|    1|2011| d_3|         NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-02-01|   11101|  Tuesday|   4|    2|2011| d_4|         NULL|        NULL|        NULL|        NULL|      1|      1|      0|
|2011-02-02|   11101|Wednesday|   5|    2|2011| d_5|         NULL|        NULL|        NUL

(None, None)

In [6]:
df = sales.join(
    calendar,
    sales.day == calendar.d,
    how="left"
)

In [7]:
df = df.select(
    "id",
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "sales",
    "date",
    "wm_yr_wk",
    "wday",
    "month",
    "year",
    "event_name_1"
)

In [8]:
df = df.join(
    sell_prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

In [9]:
df = df.withColumn(
    "is_event",
    when(col("event_name_1").isNotNull(), 1).otherwise(0)
)

In [10]:
df = df.fillna({"sell_price": 0})

In [ ]:
df_daily = df.select(
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "date",
    "wday",
    "month",
    "year",
    "sell_price",
    "is_event",
    "sales"
)

```py
df_daily.count()
```
>58327370

In [25]:
df_daily.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- cat_id: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- wday: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- sell_price: double (nullable = false)
 |-- is_event: integer (nullable = false)
 |-- sales: integer (nullable = true)



In [30]:
df_agg = df_daily.groupBy(
    "store_id",
    "dept_id",
    "month",
    "year"
).agg(
    avg("sales").alias("avg_sales"),
    avg("sell_price").alias("avg_price"),
    sum("is_event").alias("event_count")
)

df_agg.printSchema()
df_agg.show(5)


root
 |-- store_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- avg_sales: double (nullable = true)
 |-- avg_price: double (nullable = true)
 |-- event_count: long (nullable = true)

+--------+-----------+-----+----+-------------------+------------------+-----------+
|store_id|    dept_id|month|year|          avg_sales|         avg_price|event_count|
+--------+-----------+-----+----+-------------------+------------------+-----------+
|    CA_1|    FOODS_3|    3|2011| 1.8537608278132716| 1.348164073217579|       3292|
|    CA_1|  HOBBIES_1|    3|2012| 0.9685173697270472|3.3286104218362325|        832|
|    CA_1|HOUSEHOLD_2|    5|2015| 0.4511118070779831| 5.763180081428109|       1545|
|    CA_2|HOUSEHOLD_2|    4|2013|0.48815533980582526| 4.848798705501607|        515|
|    CA_3|    FOODS_2|    4|2012|  1.533249581239531| 2.680425460636521|       1194|
+--------+-----------+-----+----+----

In [31]:
kategorikal = ["store_id", "dept_id"]

df = df_agg

for k in kategorikal:
    indexer = StringIndexer(
        inputCol=k,
        outputCol=f"{k}_idx",
        handleInvalid="keep"
    )
    df = indexer.fit(df).transform(df)

df.printSchema()


root
 |-- store_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- avg_sales: double (nullable = true)
 |-- avg_price: double (nullable = true)
 |-- event_count: long (nullable = true)
 |-- store_id_idx: double (nullable = false)
 |-- dept_id_idx: double (nullable = false)



In [32]:
fitur = [
    "store_id_idx",
    "dept_id_idx",
    "month",
    "year",
    "avg_price",
    "event_count"
]

assembler = VectorAssembler(
    inputCols=fitur,
    outputCol="features"
)

df_final = assembler.transform(df)


In [33]:
df_final.select("features", "avg_sales").show(5, truncate=False)


+---------------------------------------------+-------------------+
|features                                     |avg_sales          |
+---------------------------------------------+-------------------+
|[0.0,2.0,3.0,2011.0,1.348164073217579,3292.0]|1.8537608278132716 |
|[0.0,3.0,3.0,2012.0,3.3286104218362325,832.0]|0.9685173697270472 |
|[0.0,6.0,5.0,2015.0,5.763180081428109,1545.0]|0.4511118070779831 |
|[1.0,6.0,4.0,2013.0,4.848798705501607,515.0] |0.48815533980582526|
|[2.0,1.0,4.0,2012.0,2.680425460636521,1194.0]|1.533249581239531  |
+---------------------------------------------+-------------------+
only showing top 5 rows



In [36]:
# df_final.count()

In [34]:
df_final.select("features", "avg_sales") \
    .write \
    .mode("overwrite") \
    .parquet("../dataset/cooked/feature_sales")
